In [42]:
from datetime import datetime
from src import config
import hopsworks
import pandas as pd
import logging
import mlflow
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import sys
from pathlib import Path

# Añade src al path para importar los módulos
sys.path.append(str(Path().resolve().parent / 'src'))

In [43]:
%reload_ext autoreload
%autoreload 2

In [44]:
# Configuración básica de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger('feature_view_creation')

In [ ]:
# Conectar a Hopsworks y al Feature Store
try:
    # Login y conexión al proyecto
    project = hopsworks.login(
        api_key_value=config.HOPSWORKS_API_KEY, 
        project=config.HOPSWORKS_PROJECT_NAME)
    
    # Conexión al feature store
    feature_store = project.get_feature_store()
    
    # Conexión al feature group
    feature_group = feature_store.get_feature_group(
        name=config.FEATURE_GROUP_NAME,
        version=config.FEATURE_GROUP_VERSION
    )
    
    logger.info(f"Conexión exitosa al Feature Group: {feature_group.name} (v{feature_group.version})")
    
except Exception as e:
    logger.error(f"Error en conexión: {e}")
    raise

In [46]:
# Crear/obtener feature view con características seleccionadas
try:
    # Características específicas a incluir
    selected_features = ['familia', 'base_imponible', 'is_summer_peak', 'is_easter', 'week_start']
    feature_view_name = config.FEATURE_VIEW_NAME
    feature_view_version = 1
    
    # Intentar obtener la feature view existente primero
    try:
        feature_view = feature_store.get_feature_view(
            name=feature_view_name,
            version=feature_view_version
        )
        logger.info(f"Feature view existente recuperada: {feature_view.name} (v{feature_view.version})")
    
    except:
        # Si no existe, crear una nueva
        # Obtener objetos Feature para las características seleccionadas
        selected_feature_objects = [f for f in feature_group.features if f.name in selected_features]
        
        # Crear query con características seleccionadas
        specific_query = feature_group.select(selected_feature_objects)
        
        # Crear la feature view
        feature_view = feature_store.create_feature_view(
            name=feature_view_name,
            version=feature_view_version,
            query=specific_query,
            description=f"Feature view con características: {', '.join(selected_features)}"
        )
        logger.info(f"Nueva feature view creada: {feature_view.name} (v{feature_view.version})")
    
except Exception as e:
    logger.error(f"Error al crear/obtener feature view: {e}")
    raise

2025-08-23 19:26:39,528 INFO: Feature view existente recuperada: times_series_bolleria_feature_view (v1)


In [ ]:
# Obtener datos de la feature view
try:
    # Obtener datos en batch normal
    df_ts = feature_view.get_batch_data()
    
    # Mostrar resumen de los datos obtenidos
    logger.info(f"Datos obtenidos: {df_ts.shape[0]} filas, {df_ts.shape[1]} columnas")
    logger.info(f"Columnas disponibles: {list(df_ts.columns)}")
    print("Muestra de datos:")
    print(df_ts.head(3))
    
except Exception as e:
    logger.error(f"Error al obtener datos: {e}")
    raise

In [ ]:
# Obtener datos de entrenamiento (training_data)
try:
    # Obtener datos de entrenamiento (X, y) desde la feature view
    df_ts = feature_view.training_data()

except Exception as e:
    print(f"Error al obtener datos de entrenamiento: {e}")

In [ ]:
# Procesar datos para entrenamiento
from src.data_utils import transformar_features_target

try:
       
    # Procesar datos usando la función mejorada que acepta tuplas directamente
    features_and_target = transformar_features_target(
        df_ts,
        lags_list=[1, 2, 3, 52], 
        columna_target='base_imponible',
        cols_exogenas=['is_easter', 'is_summer_peak'],
        periodos_adelante=1,
        eliminar_nulos=True,
        return_format='dataframe'  # Obtenemos un único DataFrame con features y target
    )
    
    # Mostrar información de los datos procesados
    logger.info(f"Datos procesados: {features_and_target.shape[0]} filas, {features_and_target.shape[1]} columnas")
    logger.info(f"Variables disponibles: {list(features_and_target.columns)}")
    print("\nMuestra de datos procesados:")
    print(features_and_target.head(3))
    
except Exception as e:
    logger.error(f"Error al procesar datos: {e}")
    raise

In [ ]:
# Split temporal (automático 80/20 ya incluido en la función)
from src.data_split import train_test_split

try:
    X_train, y_train, X_test, y_test = train_test_split(
        features_and_target,
        target='target'  # o 'base_imponible' según tu pipeline
        # split_ratio=0.8  # puedes cambiar el porcentaje si lo necesitas
    )
    print(f"Train: {X_train.shape}, Test: {X_test.shape}")
except Exception as e:
    logger.error(f"Error en el split temporal: {e}")
    raise


In [ ]:
# Configuración de MLflow para registrar todo en el servidor web y visualizar en la UI
mlflow.set_tracking_uri('http://127.0.0.1:5000')
mlflow.set_experiment("fleca_bolleria_v2")

In [53]:
# Eliminar columnas datetime antes de entrenar
for df in [X_train, X_test]:
    if 'week_start' in df.columns:
        df.drop('week_start', axis=1, inplace=True)

In [ ]:
# Importar XGBoost y métricas
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Definimos los modelos a entrenar
with mlflow.start_run(run_name="xgboost"):
    # Entrenamos el modelo
    model = XGBRegressor()
    model.fit(X_train, y_train)

    # Realizamos predicciones
    predictions = model.predict(X_test)

    # Calcular métricas
    mae = mean_absolute_error(y_test, predictions)
    mse = mean_squared_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)

    # Log de los parámetros
    mlflow.log_param("model", "xgboost")
    for param, value in model.get_params().items():
        mlflow.log_param(param, value)

    # Log de las métricas
    mlflow.log_metric("mae", mae)
    mlflow.log_metric("mse", mse)
    mlflow.log_metric("r2", r2)

    # Log del modelo
    mlflow.xgboost.log_model(model, "model")

    # Imprimir resultados
    print(f"MAE: {mae}")
    print(f"MSE: {mse}")
    print(f"R2: {r2}")

In [55]:
# Definimos los modelos a entrenar

models ={
    "LinearRegresion": LinearRegression(),
    "Ridge": Ridge(alpha=1.0),
    "Lasso": Lasso(alpha=0.1),
    "RandomForest": RandomForestRegressor(n_estimators=100),
    "GradientBoosting": GradientBoostingRegressor(n_estimators=100)
    }

In [ ]:
# Entrenamos y evaluamos varios modelos a la vez
for model_name, model in models.items():
    with mlflow.start_run(run_name=model_name):
        # Entrenamos el modelo
        model.fit(X_train, y_train)
        # Realizamos predicciones
        y_pred = model.predict(X_test)

        # Calculamos las métricas
        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        # Registramos las métricas en mlflow
        mlflow.log_metrics({
            "mae": mae,
            "mse": mse,
            "r2": r2_score(y_test, y_pred)
        })
        # Registramos los hiperparámetros en mlflow
        mlflow.log_params({
            "model": model_name,
            "n_estimators": model.n_estimators if hasattr(model, "n_estimators") else None,
            "max_depth": model.max_depth if hasattr(model, "max_depth") else None,
            "learning_rate": model.learning_rate if hasattr(model, "learning_rate") else None
        })

        # Registramos el modelo en mlflow
        mlflow.sklearn.log_model(model, "model")

        logger.info(f"Modelo: {model_name} - MAE: {mae}, MSE: {mse}, R2: {r2}")


In [ ]:
# Buscar el mejor modelo por menor MAE y registrar automáticamente en MLflow Model Registry
import mlflow
from mlflow.tracking import MlflowClient
client = MlflowClient()
experiment = client.get_experiment_by_name('fleca_bolleria_v2')
runs = client.search_runs(experiment_ids=[experiment.experiment_id], order_by=['metrics.mae ASC'])
best_run = runs[0]  # El de menor MAE
print('Mejor modelo:', best_run.data.params['model'])
print('Run ID del mejor modelo:', best_run.info.run_id)


In [ ]:
# Registramos el mejor modelo
# Id del run del mejor modelo (puedes cogerlo de la UI)
run_id = "062e601e74e9491cac10fa8dd90f2cff"  # Este es el run_id real
model_uri = f'runs:/{run_id}/model'  # Sintaxis correcta para MLflow
model_description = 'Modelo de Random Forest entrenado con 1,2,3,52 Lags'
mlflow.register_model(model_uri, 'RandomForest')

# ESte no funciona (hay que investigar por qué)

In [61]:
# Entrenamiento y evaluación con XGBoost
from src.model import train_evaluate_xgboost

# Eliminar columnas datetime antes de entrenar
for df in [X_train, X_test]:
    if 'week_start' in df.columns:
        df.drop('week_start', axis=1, inplace=True)

with mlflow.start_run(run_name="xgboost"):
    resultados = train_evaluate_xgboost(X_train, y_train, X_test, y_test)
    model = resultados["model"]

    # Log de los parámetros del modelo
    mlflow.log_param("model", "xgboost")
    for param, value in model.get_params().items():
        mlflow.log_param(param, value)

    # Log de las métricas
    mlflow.log_metric("mae", resultados["mae"])
    
    mlflow.log_metric("r2", resultados["r2"])

    # Log del modelo
    mlflow.xgboost.log_model(model, "model")

    # Imprimir resultados
    print(f"MAE: {resultados['mae']}")
   
    print(f"R2: {resultados['r2']}")


2025/08/23 12:24:56 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/23 12:24:58 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/08/23 12:24:58 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


MAE: 236.2600927734375
R2: 0.08752420359999014
🏃 View run xgboost at: http://localhost:5000/#/experiments/1/runs/65ddfe3dd1954b21a34ab0388987d4e4
🧪 View experiment at: http://localhost:5000/#/experiments/1


In [18]:
# Ejemplo: cargar el modelo registrado desde el Model Registry
import mlflow

# Cambia 'RandomForest' y versión si corresponde
model_uri = 'models:/RandomForest/1'
model = mlflow.pyfunc.load_model(model_uri)

# Realizar predicciones (ejemplo con X_test)
# y_pred = model.predict(X_test)

print('Modelo cargado desde el Model Registry:', model_uri)

Modelo cargado desde el Model Registry: models:/RandomForest/1
